In [18]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from IPython import display
display.set_matplotlib_formats('svg')

/var/folders/nn/4_714bpd2qz7gm9d27_r3qw40000gn/T/ipykernel_22036/1910417384.py:8: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


In [46]:
import seaborn as sns
iris = sns.load_dataset('iris')

In [47]:
# organize the data

# convert pandas dataframe into pytorch tensor
data = torch.tensor(iris[iris.columns[0:4]].values).float()

# transform species to numbers

labels = torch.zeros(len(data), dtype = torch.long)
labels[iris.species == 'versicolor'] = 1
labels[iris.species == 'virginica'] = 2

labels.shape

torch.Size([150])

In [48]:
propTraining = 0.8
nTraining = int(len(labels) * propTraining)
traintestBool = np.zeros(len(labels), dtype = bool)
items2use4train = np.random.choice(range(len(labels)), nTraining, replace = False)
traintestBool[items2use4train] = True
traintestBool

array([ True,  True,  True, False, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True, False, False,  True, False, False,  True,
       False,  True, False,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False, False,

In [49]:
print(data.shape)
print(data[traintestBool, :].shape)
print(data[~traintestBool, :].shape)

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


In [50]:
X_train, X_test, y_train, y_test = data[traintestBool, :], data[~traintestBool, :], labels[traintestBool ], labels[~traintestBool ]

In [51]:
# create model

class ANNiris(nn.Module):
    def __init__(self, nUnits, nLayers):
        super().__init__()
        
        self.layers = nn.ModuleDict()
        self.nLayers = nLayers
        
        # input layer
        self.layers['input'] = nn.Linear(4, nUnits)
        
        # hidden layer
        for i in range(nLayers):
            self.layers[f'hidden {i}'] = nn.Linear(nUnits, nUnits)
        
        # output layer
        self.layers['output'] = nn.Linear(nUnits, 3)
        
    def forward(self, x):
        x = self.layers['input'](x)
        for i in range(self.nLayers):
            x = F.relu(self.layers[f'hidden {i}'](x))
        
        x = self.layers['output'](x)
        return x

In [23]:
data = torch.randn(10, 4)
net = ANNiris(2, 3)
net

ANNiris(
  (layers): ModuleDict(
    (input): Linear(in_features=4, out_features=2, bias=True)
    (hidden 0): Linear(in_features=2, out_features=2, bias=True)
    (hidden 1): Linear(in_features=2, out_features=2, bias=True)
    (hidden 2): Linear(in_features=2, out_features=2, bias=True)
    (output): Linear(in_features=2, out_features=3, bias=True)
  )
)

In [55]:
numepochs = 500
def trainModel(model):
    lossfun = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
    
    losses = torch.zeros(numepochs)
    
    for epochi in range(numepochs):
        yHat = model(X_train)
        
        loss = lossfun(yHat, y_train)
        losses[epochi] = loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    predictions = model(X_train)
    predlabels = torch.argmax(predictions, axis = 1)
    acc = 100 * torch.mean((predlabels == y_train).float())
    
    return acc

In [62]:
model = ANNiris(12, 3)
acc = trainModel(model)
print(f'Train Accuracy is: {acc:.2f}')

Train Accuracy is: 98.33


In [63]:
predictions = model(X_test)
predlabels = torch.argmax(predictions, axis = 1)
acc = 100 * torch.mean((predlabels == y_test).float())
print(f'Test accuracy is: {acc:.2f}')

Test accuracy is: 100.00
